# Cleaning - Simply Recipes Ingredients List

Sources: 
* https://htmlpreview.github.io/?https://github.com/kulsoom-abdullah/kulsoom-abdullah.github.io/blob/master/AWS-lambda-implementation/model_implementation/recipe%20binary%20classification/recipe%20binary%20classification.html#Words-to-Remove
* https://towardsdatascience.com/https-towardsdatascience-com-end-to-end-recipe-cuisine-classification-e97f4ac22104


NYT - Natural Language Processor for Ingredients 

* https://github.com/nytimes/ingredient-phrase-tagger
* https://homepages.inf.ed.ac.uk/csutton/publications/crftutv2.pdf

CRF Tutorial 
* https://www.analyticsvidhya.com/blog/2018/08/nlp-guide-conditional-random-fields-text-classification/
* http://acepor.github.io/2017/03/06/CRF-Python/
* http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/
* https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html

In [61]:
import pandas as pd
import numpy as np
import re
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/markishab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [62]:
recipes_df = pd.read_pickle('../../data/02_intermediate/recipies_sr_inter.pkl')

In [63]:
recipes_df.reset_index(inplace=True, drop=True)

In [93]:
recipes_df.sort_values(by=['ingredients'])

,title,prep_time,cook_time,recipe_yield,tags,ingredients,byline,link_food
953,NaN,NaN,NaN,NaN,"['Recipe Collections', 'Budget']",[],Summer Miller,https://www.simplyrecipes.com/collection/5_eas...
1134,3 Ways to Make a Great Cheesecake,NaN,NaN,NaN,['Baking'],[],Carrie Havranek,https://www.simplyrecipes.com/cheesecake_recip...
966,15 Great Meals to Make with Canned Tuna,NaN,NaN,NaN,"['Recipe Collections', 'Dinner', 'Budget']",[],Elise Bauer,https://www.simplyrecipes.com/great_meals_to_m...
954,How to Store and Freeze Banana Bread,NaN,NaN,NaN,"['Tips', 'Banana Bread', 'Baking', 'Budget', '...",[],Summer Miller,https://www.simplyrecipes.com/how_to_store_and...
1212,The Difference Between Baking Soda and Baking ...,NaN,NaN,NaN,"['Tips', 'Baking', 'How To']",[],Elise Bauer,https://www.simplyrecipes.com/the_difference_b...
33,6 Different Meals to Make With Pulled Pork,NaN,NaN,NaN,"['Tips', 'Pork', 'Pulled Pork', 'Dinner']",[],Summer Miller,https://www.simplyrecipes.com/6_different_meal...
1154,4 Tips for Making the Best Cinnamon Rolls Ever,NaN,NaN,NaN,"['Tips', 'Baking', 'How To']",[],Summer Miller,https://www.simplyrecipes.com/4_tips_for_makin...
1290,6 Crowd-Pleasing Recipes with Cheese,NaN,NaN,NaN,"['Cheese', 'Breakfast and Brunch', 'Dessert', ...",[],Simply Recipes Team,https://www.simplyrecipes.com/6_crowd_pleasing...
889,NaN,NaN,NaN,NaN,"['Recipe Collections', 'Pasta and Noodles', 'F...",[],Carrie Havranek,https://www.simplyrecipes.com/collection/12_fr...
190,What’s the Difference Between Stock and Broth?,NaN,NaN,NaN,"['Ingredient Guides', 'Tips', 'Chicken', 'Chic...",[],Emma Christensen,https://www.simplyrecipes.com/whats_the_differ...


Let's drop the N/A columns

In [110]:
recipes_df.dropna(inplace=True)

## let's clean the list so that it can be used in a crf model 

In [111]:
ingredients_lists = list(recipes_df.ingredients)

In [112]:
len(ingredients_lists)

1177

In [113]:
ingredient_list_new = []
for recipe in ingredients_lists:
    sub_list = []
    for ingredient in recipe:
        ingredient_list_new.append(ingredient.split(','))

Let's isolate the actual ingredients

In [128]:
for element in ingredient_list_new:
    try:
        while(" " in element) : 
            element.remove(" ")
    except:
        pass

Replace ```'\\\\xa0'``` with space

In [129]:
new_list = []
for ingr_list in ingredient_list_new:
    ingr_list = [word.replace('\\\\xa0', ' ') for word in ingr_list]
    new_list.append(ingr_list)

In [130]:
len(new_list)

1177

In [131]:
final_list = []
for il in new_list:
    il = [word.replace('\\xa0', ' ') for word in il]
    final_list.append(il)

In [132]:
len(final_list)

1177

Let's figure out the final list

In [133]:
for recipe in final_list:
    for idx, ingredient in enumerate(recipe):
        if ingredient.startswith('For'):
            recipe.pop(idx)

In [134]:
for recipe in final_list:
    for idx, ingredient in enumerate(recipe):
        if ingredient.startswith('For'):
            print(ingredient)

In [135]:
len(final_list)

1177

In [136]:
recipes_df.ingredients[70]

["1 tablespoons extra virgin olive oil', '1 yellow onion,\\\\xa0finely chopped (about 1\\\\xa01/2 to 2 cups)', '2 cloves garlic, minced (about 2 teaspoons)', '4 ounces (110 g) Italian sweet sausage (or one 7 to 8 inch link)', '4 ounces (110 g) \\\\xa0Italian hot and spicy sausage\\\\xa0(or one 7 to 8 inch link)', '1\\\\xa0large (28 oz, 794 g) can of whole peeled tomatoes', '1 pound (16 oz, 450 g) spaghetti pasta', 'Salt"]

In [143]:
final_list[70]

"1 tablespoon extra virgin olive oil'"

Make titles into list 

In [138]:
food_titles = list(recipes_df.title)

In [139]:
recipe_dict = dict()
for title, recipes in zip(food_titles, final_list):
#     print(title)
    recipe_dict[title] = recipes

In [140]:
len(recipes_df)

1177

In [144]:
recipes_df['ingredients'] = final_list

In [145]:
recipes_df

,title,prep_time,cook_time,recipe_yield,tags,ingredients,byline,link_food
0,Grilled Cheese BLT,10 minutes,10 minutes,4 sandwiches,"['Dinner', 'Lunch', 'Sandwich', 'Favorite Summ...","[8 slices sourdough bread', '4 tablespoon uns...",Aaron Hutcherson,https://www.simplyrecipes.com/recipes/grilled_...
1,Pulled Pork Sandwich,10 minutes,"2 hours, 45 minutes",Serves 6 to 8,"['Dinner', 'Sandwich', 'Budget', 'Comfort Food...","[ '1 large onion, chopped', '6 garlic cloves...",Elise Bauer,https://www.simplyrecipes.com/recipes/pulled_p...
2,How to Make Bacon in the Oven,5 minutes,20 minutes,12 strips,"['Tips', 'Breakfast and Brunch', 'Baking', 'Ho...","[12 strips bacon', '1/2 teaspoon ground black...",Nick Evans,https://www.simplyrecipes.com/recipes/how_to_m...
3,Sausage Stuffed Zucchini,15 minutes,1 hour,Serves 4,"['Dinner', 'Favorite Summer', 'Make-ahead', 'I...","[2 tablespoons extra virgin olive oil', '1/2 ...",Elise Bauer,https://www.simplyrecipes.com/recipes/italian_...
5,Ginger Pork Rice Bowls,15 minutes,20 minutes,4-6 servings,"['Eat Your Food', 'Family Dinner Ideas', 'Dinn...","[ '1 tablespoon olive oil', '1/2 cup diced wh...",Nick Evans,https://www.simplyrecipes.com/recipes/ginger_p...
6,Hawaiian SPAM Tacos with Pineapple,10 minutes,30 minutes,"12 small tacos, serving 4 adults","['Eat Your Food', 'Dinner', 'Kid-friendly', 'H...","[DAD ADD: Haba', 'ñero Guacamole', '1 habañe...",Nick Evans,https://www.simplyrecipes.com/recipes/hawaiian...
7,Air Fryer Chinese Egg Rolls,20 minutes,25 minutes,12 egg rolls,"['Appetizer', 'Snack', 'Air Fryer', 'Quick and...","[ '1 tablespoon olive oil', '1 pound ground p...",Nick Evans,https://www.simplyrecipes.com/recipes/air_frye...
8,Grilled Bacon-Wrapped Stuffed Hot Dogs,10 minutes,11 minutes,Makes 4 stuffed hot dogs,"['Dinner', 'Favorite Summer', 'Grill', 'Bacon'...","[1 teaspoon ketchup', '1 teaspoon Dijon musta...",Elise Bauer,https://www.simplyrecipes.com/recipes/grilled_...
9,Italian Grilled Cheese Sandwiches,10 minutes,20 minutes,4 sandwiches,"['Eat Your Food', 'Dinner', 'Sandwich', 'Kid-f...","[ 'Olive and Cauliflower Giardiniera', '1 cup...",Nick Evans,https://www.simplyrecipes.com/recipes/italian_...
11,Biscuits and Gravy Casserole,25 minutes,35 minutes,12 servings,"['Breakfast and Brunch', 'Casserole', 'Comfort...",[ '2 1/4 cups (315 g) all-purpose flour (see R...,Irvin Lin,https://www.simplyrecipes.com/recipes/biscuits...


In [146]:
recipes_df.to_pickle('../../data/02_intermediate/recipes_sr_final.pickle')